In [35]:
import scanpy as sc
import numpy as np
import random

In [111]:
ipf = sc.read('../../../pipeline/data/analysis/hlca_healthy_ipf_embed_balanced/ipf_random_subset10k_seacells.h5ad')
ipf

AnnData object with n_obs × n_vars = 10000 × 30
    obs: "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ancestry', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'assay', 'cause_of_death', 'cell_type', 'core_or_extension', 'dataset', 'development_stage', 'disease', 'donor_id', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'sex', 'smoking_status', 'study', 'subject_type', 'suspension_type', 'tissue', 'tissue_coarse_unharmonized', 'tissue_detailed_unharmonized', 'tissue_dissociation_protocol', 'tissue_level_2', 'tissue_level_3', 'tissue_sampling_method', 'total_counts', 'transf_ann_level_1_label', 'transf_ann_level

In [112]:
sc.pp.neighbors(ipf, use_rep='X', n_neighbors=50)

In [113]:
ipf.obsp['connectivities']

<10000x10000 sparse matrix of type '<class 'numpy.float32'>'
	with 698708 stored elements in Compressed Sparse Row format>

In [138]:
n_meta_cells = 200

In [139]:
meta_cells_idx = random.sample(list(enumerate(ipf.obs_names)), n_meta_cells)
meta_cells_idx[:5]

[(6456, 'GCCAAATCATGACGGA_IPF1_tsukui'),
 (7478, 'CGACTTCAGCTTATCG_IPF2_tsukui'),
 (5340, '209I-a_GTGCAGCGTCTCTTTA_adams'),
 (7590, 'AACTCAGTCTGTTGAG_IPF1_tsukui'),
 (185, 'ILD59-1_TCAGGTACACATTTCT_haberman')]

In [140]:
indices = [t[0] for t in meta_cells_idx]
indices[:5]

[6456, 7478, 5340, 7590, 185]

In [141]:
conn = ipf.obsp['connectivities']
conn    

<8246x8246 sparse matrix of type '<class 'numpy.float32'>'
	with 514046 stored elements in Compressed Sparse Row format>

In [142]:
ipf.obs['metacells'] = np.nan
for i, idx in enumerate(indices):
    tmp = conn[idx, :]
    nonzero_idx = np.where(tmp.A != 0)[1]
    neighbor_cells_names = list(ipf.obs.iloc[nonzero_idx].index)
    ipf.obs.loc[neighbor_cells_names, 'metacells'] = f'metacell-{i}'

In [143]:
ipf = ipf[~ipf.obs['metacells'].isna()].copy()
ipf

AnnData object with n_obs × n_vars = 6374 × 30
    obs: "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ancestry', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'assay', 'cause_of_death', 'cell_type', 'core_or_extension', 'dataset', 'development_stage', 'disease', 'donor_id', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'sex', 'smoking_status', 'study', 'subject_type', 'suspension_type', 'tissue', 'tissue_coarse_unharmonized', 'tissue_detailed_unharmonized', 'tissue_dissociation_protocol', 'tissue_level_2', 'tissue_level_3', 'tissue_sampling_method', 'total_counts', 'transf_ann_level_1_label', 'transf_ann_level_

In [144]:
ipf.obs[['metacells']].groupby('metacells').value_counts()

metacells
metacell-0      11
metacell-1       2
metacell-10     16
metacell-100    57
metacell-101    32
                ..
metacell-95     13
metacell-96     15
metacell-97     15
metacell-98     20
metacell-99     50
Name: count, Length: 199, dtype: int64

In [145]:
np.min(ipf.obs[['metacells']].groupby('metacells').value_counts())

1